# DEC Supervised Clustering

This notebook focuses on using some kind of supervision for helping the clustering. This includes different techniques that we discussed with Dr. Ece Kamar from Microsoft during her visit to UMN. The different techniques include: 
    1. Initializing cluster centers based on highly confident galaxy images
    2. Providing some kind of feedback on how good/bad clustering was in the previous iteration

Some other techniques that we are also trying to include and analyse are:
3. Clustering over a quadrant of the image to try to get rid of the worm shape PCA plot
4. Using different clustering techniques which include denisty and heirarchical clustering